# Calculate daily returns for a list of symbols at day 0 and create a correlation matrix

This notebook calculates daily returns at day 0 (t0/t-1) from a dataframe consists of hundreds of stock symbols' historical prices, and creates a new dataframe with calculated t0 returns, and then creates a correlation matrix for these returns

In [1]:
import pandas as pd
import os
import numpy

def ends(df,x=1):
    dfends=df.head(x).append(df.tail(x))
    return dfends 

In [2]:
# Read the cleaned data and sort by symbol and by data, to make sure the date is in right order for any analysis

data_df=pd.read_csv(os.path.join('C:/Users/jack.jin/Documents/Market', 'SP500_FIVE_YR_Cleaned_Sorted.csv'),sep=',')

In [3]:
ends(data_df,2)

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Row Count
0,2015-01-02,41.180000,41.310001,40.369999,40.560001,38.733665,1529200.0,A,1246
1,2015-01-05,40.320000,40.459999,39.700001,39.799999,38.007896,2041800.0,A,1246
609292,2019-12-11,123.199997,123.709999,122.809998,123.559998,123.559998,1803000.0,ZTS,1246
609293,2019-12-12,123.690002,125.000000,123.389999,123.599998,123.599998,1069232.0,ZTS,1246


In [4]:
#https://pandas.pydata.org/pandas-docs/stable/user_guide/computation.html#expanding-windows

# Define Func to calculate daily changes:
def stock_daily_change(df):
    
    # Get the unique symbol list
    #symbol_list = list(set(stock_df['Symbol'])) this line works too but it will change the order.
    symbol_list = pd.unique(df['Symbol']).tolist()
    
    # Create an empty dataframe to host to-be processed data
    combo_df=pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol','Row Count','Adj Close Change'])
    
    for sym in symbol_list:
        
        indiv_sym_df=df.loc[df['Symbol'] == sym,]
        adj_cl_pct=pd.DataFrame.pct_change(indiv_sym_df[['Adj Close']]) 
        adj_cl_pct=adj_cl_pct.rename(columns={'Adj Close':'Adj Close Change'})
        
        indiv_sym_df=indiv_sym_df.join(adj_cl_pct)
        combo_df=combo_df.append(indiv_sym_df)
    
    return combo_df 

In [5]:
# Apply the func to calculate daily changes:

stock_with_pct=stock_daily_change(data_df)

In [6]:
ends(stock_with_pct,2)

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Row Count,Adj Close Change
0,2015-01-02,41.180000,41.310001,40.369999,40.560001,38.733665,1529200.0,A,1246,NaN
1,2015-01-05,40.320000,40.459999,39.700001,39.799999,38.007896,2041800.0,A,1246,-0.018737
609292,2019-12-11,123.199997,123.709999,122.809998,123.559998,123.559998,1803000.0,ZTS,1246,0.006353
609293,2019-12-12,123.690002,125.000000,123.389999,123.599998,123.599998,1069232.0,ZTS,1246,0.000324


In [7]:
# Drop the first value (NaN) 

stock_with_pct.dropna(how='any', inplace=True)

In [8]:
# NaN dropped

ends(stock_with_pct,2)

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Row Count,Adj Close Change
1,2015-01-05,40.320000,40.459999,39.700001,39.799999,38.007896,2041800.0,A,1246,-0.018737
2,2015-01-06,39.810001,40.020000,39.020000,39.180000,37.415798,2080600.0,A,1246,-0.015578
609292,2019-12-11,123.199997,123.709999,122.809998,123.559998,123.559998,1803000.0,ZTS,1246,0.006353
609293,2019-12-12,123.690002,125.000000,123.389999,123.599998,123.599998,1069232.0,ZTS,1246,0.000324


In [9]:
stock_with_pct.to_csv(os.path.join('C:/Users/jack.jin/Documents/Market', 'SP500_FIVE_YR_Cleaned_Sorted_t0_PCT.csv'),sep=',', index=False)

In [10]:
def create_return_df(df):
    
    symbol_list = pd.unique(df['Symbol']).tolist()
    
    column_combo_df=pd.DataFrame(pd.unique(df['Date']), columns=['Date']).sort_values(['Date'])
    
    for sym in symbol_list:
        
        indiv_sym_df=df.loc[df['Symbol'] == sym,['Date','Adj Close Change']]
        indiv_sym_df.rename(columns={'Adj Close Change': sym}, inplace=True)
        
        column_combo_df=pd.merge(column_combo_df, indiv_sym_df,on='Date',how='left')
        
        
    return column_combo_df

return_df=create_return_df(stock_with_pct)
ends(return_df)

,Date,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
0,2015-01-05,-0.018737,-0.000557,-0.013181,-0.028172,-0.018819,-0.008512,-0.006433,0.000223,-0.016884,...,-0.011351,-0.018348,-0.027362,-0.006932,-0.021818,-0.062238,-0.020318,0.037304,-0.037469,-0.006003
1244,2019-12-12,0.010249,0.031646,0.000257,-0.002585,0.014992,0.000346,-0.001733,0.006713,0.005437,...,-0.014398,0.020880,0.014864,-0.002113,0.013007,0.006313,0.006738,-0.001258,0.026585,0.000324


In [14]:
# Drop the Date column and create a correlation matrix

return_df.drop('Date',1,inplace=True)
ret_corr=return_df.corr()
ends(ret_corr,4)

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
A,1.000000,0.372553,0.207878,0.445141,0.402519,0.286600,0.299968,0.595891,0.553744,0.524865,...,0.149043,0.410438,0.395078,0.351388,0.387616,0.519608,0.365788,0.452897,0.438263,0.486988
AAL,0.372553,1.000000,0.208654,0.340233,0.222044,0.249498,0.178961,0.364239,0.343524,0.352580,...,0.013801,0.379700,0.225104,0.231519,0.320794,0.357178,0.266028,0.278171,0.376341,0.305783
AAP,0.207878,0.208654,1.000000,0.176067,0.178450,0.168699,0.131780,0.247762,0.235082,0.177925,...,0.115708,0.166683,0.206631,0.151880,0.215371,0.233242,0.219150,0.213161,0.281693,0.202132
AAPL,0.445141,0.340233,0.176067,1.000000,0.266364,0.218473,0.315407,0.435026,0.474030,0.498224,...,0.087700,0.432493,0.364162,0.264042,0.310439,0.388688,0.319484,0.325192,0.324113,0.370652
YUM,0.365788,0.266028,0.219150,0.319484,0.224449,0.175870,0.194788,0.375082,0.402187,0.423007,...,0.204245,0.277842,0.271857,0.186630,0.254951,0.332065,1.000000,0.280024,0.302180,0.349278
ZBH,0.452897,0.278171,0.213161,0.325192,0.368375,0.280400,0.218303,0.545098,0.404341,0.393679,...,0.151441,0.270667,0.323698,0.347943,0.305842,0.349728,0.280024,1.000000,0.319914,0.379576
ZION,0.438263,0.376341,0.281693,0.324113,0.288227,0.240414,0.190629,0.377045,0.405153,0.308760,...,-0.102242,0.335144,0.419580,0.239297,0.392274,0.459191,0.302180,0.319914,1.000000,0.280372
ZTS,0.486988,0.305783,0.202132,0.370652,0.386446,0.298774,0.288482,0.508521,0.463309,0.425151,...,0.152822,0.300779,0.299449,0.311488,0.311146,0.403668,0.349278,0.379576,0.280372,1.000000


In [15]:
# mask on lower-triangle only
# https://stackoverflow.com/questions/31750595/select-values-from-any-column-row-based-on-criteria
import numpy as np

mask = np.ones_like(ret_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = False
mask

array([[False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True,  True, False, ..., False, False, False],
       ...,
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ...,  True, False, False],
       [ True,  True,  True, ...,  True,  True, False]])

In [19]:
# reshape the correlation matrix
corr_stacked = ret_corr.stack()

In [20]:
ends(corr_stacked,10)

A    A       1.000000
     AAL     0.372553
     AAP     0.207878
     AAPL    0.445141
     ABBV    0.402519
     ABC     0.286600
     ABMD    0.299968
     ABT     0.595891
     ACN     0.553744
     ADBE    0.524865
ZTS  XEL     0.152822
     XLNX    0.300779
     XOM     0.299449
     XRAY    0.311488
     XRX     0.311146
     XYL     0.403668
     YUM     0.349278
     ZBH     0.379576
     ZION    0.280372
     ZTS     1.000000
dtype: float64

In [21]:
high_corr=corr_stacked[(corr_stacked > 0.7) & (mask.ravel())]

In [25]:
ends(high_corr,20)

AEP   AEE     0.838459
AMP   AMG     0.709580
AON   AJG     0.719244
ATO   AEE     0.771910
      AEP     0.762548
AVB   AIV     0.846236
      ARE     0.704225
AWK   AEE     0.736012
      AEP     0.756278
      ATO     0.735286
BAC   AMP     0.738680
BEN   AMG     0.727181
      AMP     0.724837
BK    BAC     0.751776
BLK   AMG     0.734285
      AMP     0.736883
      BEN     0.745230
BWA   APTV    0.712673
BXP   ARE     0.718212
      AVB     0.718556
ZION  CMA     0.872929
      FITB    0.839348
      GS      0.708795
      HBAN    0.817473
      JPM     0.780090
      KEY     0.825075
      LNC     0.763332
      MET     0.710003
      MS      0.757973
      MTB     0.800820
      NTRS    0.744998
      PBCT    0.809852
      PNC     0.812511
      PRU     0.742276
      RF      0.874888
      RJF     0.725026
      SCHW    0.732458
      SIVB    0.757380
      USB     0.783568
      WFC     0.725722
dtype: float64